# Lab 2: Data Engineering
## Exercise 1: Create a dataset

Until now, we have used the CIFAR-10 dataset from Keras. Every script started by preprocessing this dataset into a subset with the four classes that we chose. Because CIFAR-10 is quite small, this doesn't take much time, so it didn't bother us.

Imagine you have a gigantic dataset. This is quite normal when you are doing machine learning in the real world. Preprocessing it takes a long time, so we really don't want to do it for every training experiment. What's the solution? ClearML Datasets!

Let's create a dataset in ClearML for our subset of CIFAR-10. This is done by running a script to preprocess the data, write it to the file system, and then upload it to ClearML. We need to write data to files first because ClearML datasets are always created from files. You can write any Python data to files using `pickle.dump()`, or when you have NumPy arrays, use `numpy.save()`.

To create a ClearML dataset use `Dataset.create()`:

https://clear.ml/docs/latest/docs/references/sdk/dataset#datasetcreate \
https://clear.ml/docs/latest/docs/clearml_data/data_management_examples/data_man_python

Afterward, go to the ClearML website, click on DATASETS on the left side, and find the dataset you just created.

We won't do this now, but later we can use [Dataset.get()](https://clear.ml/docs/latest/docs/references/sdk/dataset/#datasetget) to download our new dataset before training.

In [10]:
import os
from dotenv import load_dotenv

#%pip install -q clearml nbconvert

%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml

load_dotenv()

CLEARML_API_ACCESS_KEY=os.getenv("CLEARML_API_ACCESS_KEY")
CLEARML_API_SECRET_KEY=os.getenv("CLEARML_API_SECRET_KEY")

if CLEARML_API_ACCESS_KEY is None:
    raise KeyError("CLEARML_API_ACCESS_KEY")

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml


In [11]:
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import numpy as np
from clearml import Dataset

# download the dataset
(images, labels), _ = keras.datasets.cifar10.load_data()

# there are 10 classes of images
all_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# choose four classes (feel free to change this!)
class_names = ["bird", "cat", "deer", "dog"]
print("Class names:", class_names)

# only keep images of these classes
class_indexes = [all_classes.index(c) for c in class_names]
to_keep = np.array([l in class_indexes for l in labels])
images = images[to_keep]
labels = labels[to_keep]

# change indexes from 10 to 2 classes
labels = np.array([class_indexes.index(l) for l in labels])

# normalize pixels between 0 and 1
images = images / 255.0

# ---
# TMP Only store 10 images/labels, because we want fast upload.
# This way we can learn the dataset SDK without waiting too long.
images = images[:10]
labels = labels[:10]
# ---

# split into train and test set
split = round(len(images) * 0.8)
train_images = images[:split]
train_labels = labels[:split]
test_images = images[split:]
test_labels = labels[split:]
print("Number of train images:", len(train_images))
print("Number of test images:", len(test_images))

# TODO save numpy arrays to disk
with open('train_images.npy', 'wb') as f:
    np.save(f, train_images)
with open('train_labels.npy', 'wb') as f:
    np.save(f, train_labels)
with open('test_images.npy', 'wb') as f:
    np.save(f, test_images)
with open('test_labels.npy', 'wb') as f:
    np.save(f, test_labels)

# TODO create ClearML dataset
dataset = Dataset.create(dataset_name="my-dataset", dataset_project="vives-mlops-workshop")
dataset.add_files('train_images.npy')
dataset.add_files('train_labels.npy')
dataset.add_files('test_images.npy')
dataset.add_files('test_labels.npy')

dataset.upload()
dataset.finalize()

Class names: ['bird', 'cat', 'deer', 'dog']
Number of train images: 8
Number of test images: 2
ClearML results page: https://app.clear.ml/projects/c91b253598c347cab986e3aa22fa3207/experiments/deb4e886c1f34a29ad00559462aec69a/output/log
ClearML dataset page: https://app.clear.ml/datasets/simple/c91b253598c347cab986e3aa22fa3207/experiments/deb4e886c1f34a29ad00559462aec69a
Uploading dataset changes (4 files compressed to 83.14 KiB) to https://files.clear.ml
File compression and upload completed: total size 83.14 KiB, 1 chunk(s) stored (average size 83.14 KiB)


True